# BART-Extended architecture

The goal of this Notebook is to evaluate how to load BART extended model


In [1]:
import transformers.models.bart.modeling_bart_edited as BartExtended
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

pretrained_bart = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs. I hope one day they start eating healthier. Maybe a plant-based diet would be enough."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [2]:
bart_extended = BartExtended.BartExtendedForConditionalGeneration(pretrained_bart.config)

In [3]:
# Generate Summary
summary_ids = pretrained_bart.generate(inputs['input_ids'], num_beams=4, max_length=20, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

['My friends are cool but they eat too many carbs. Maybe a plant-based diet']


In [7]:
# Generate Summary
summary_ids = bart_extended.generate(inputs['input_ids'], num_beams=4, max_length=20, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

TypeError: forward() got an unexpected keyword argument 'encoder_outputs'

In [3]:
def init_bart_extended_layer(bart_extended_layer, bart_layer):
    # copy weights
    bart_extended_layer.self_attn.load_state_dict(bart_layer.self_attn.state_dict())
    bart_extended_layer.self_attn_layer_norm.load_state_dict(bart_layer.self_attn_layer_norm.state_dict())
    bart_extended_layer.encoder_source_attn.load_state_dict(bart_layer.encoder_attn.state_dict())
    bart_extended_layer.encoder_source_attn_layer_norm.load_state_dict(bart_layer.encoder_attn_layer_norm.state_dict())
    bart_extended_layer.encoder_knowledge_attn.load_state_dict(bart_layer.encoder_attn.state_dict())
    bart_extended_layer.encoder_knowledge_attn_layer_norm.load_state_dict(bart_layer.encoder_attn_layer_norm.state_dict())
    bart_extended_layer.fc1.load_state_dict(bart_layer.fc1.state_dict())
    bart_extended_layer.fc2.load_state_dict(bart_layer.fc2.state_dict())
    bart_extended_layer.final_layer_norm.load_state_dict(bart_layer.final_layer_norm.state_dict())

def init_bart_extended_decoder(extended_decoder, pretrained_decoder):
    # Initializing Embedding layers
    extended_decoder.embed_tokens.load_state_dict(pretrained_decoder.embed_tokens.state_dict())
    extended_decoder.embed_positions.load_state_dict(pretrained_decoder.embed_positions.state_dict())
    
    # Initializing layers
    for extended_layer, pretrained_layer in zip(extended_decoder.layers, pretrained_decoder.layers):
        init_bart_extended_layer(extended_layer, pretrained_layer)
        
    # Initializing Layer normalization layer
    extended_decoder.layernorm_embedding.load_state_dict(pretrained_decoder.layernorm_embedding.state_dict())

In [4]:
extended_decoder = BartExtended.BartExtendedDecoder(pretrained_bart.config)
init_bart_extended_decoder(extended_decoder, pretrained_bart.model.decoder) 

In [6]:
self.encoder_source = BartExtended.BartEncoder(pretrained_bart.config)
self.encoder_source.load_state_dict(pretrained_bart.encoder.state_dict())
self.encoder_knowledge = BartExtended.BartEncoder(pretrained_bart.config)
self.encoder_knowledge.load_state_dict(pretrained_bart.encoder.state_dict())

NameError: name 'self' is not defined

In [7]:
class Bart_Extended_Model(nn.Module):
    def __init__(self, config: BartConfig):
        super().__init__(config)

        padding_idx, vocab_size = config.pad_token_id, config.vocab_size
        self.shared = nn.Embedding(vocab_size, config.d_model, padding_idx)
        
        extended_decoder = BartExtended.BartExtendedDecoder(pretrained_bart.config)
        init_bart_extended_decoder(extended_decoder, pretrained_bart.model.decoder) 

        self.encoder_source = BartExtended.BartEncoder(pretrained_bart.config)
        self.encoder_source.load_state_dict(pretrained_bart.encoder.state_dict())
        self.encoder_knowledge = BartExtended.BartEncoder(pretrained_bart.config)
        self.encoder_knowledge.load_state_dict(pretrained_bart.encoder.state_dict())
        
        self.decoder = BartExtended.BartExtendedDecoder(pretrained_bart.config)
        init_bart_extended_decoder(self.decoder, pretrained_bart.model.decoder)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


NameError: name 'nn' is not defined

In [1]:
from transformers import BartTokenizer, BartExtendedForConditionalGeneration, BartConfig

BART_Extended_model = BartExtendedForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs. I hope one day they start eating healthier. Maybe a plant-based diet would be enough."
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

ImportError: cannot import name 'BartExtendedForConditionalGeneration' from 'transformers' (unknown location)

In [2]:
# Generate Summary
summary_ids = BART_model.generate(inputs['input_ids'], num_beams=4, max_length=20, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

['My friends are cool but they eat too many carbs. Maybe a plant-based diet']


In [3]:
model = pretrained_bart.model

In [4]:
fc_kk = model.encoder_source.layers[0].fc1

AttributeError: 'BartModel' object has no attribute 'encoder_source'

In [5]:
fc_kk.weight

NameError: name 'fc_kk' is not defined

In [15]:
pretrained_bart.model.encoder_knowledge.layers[0].fc1.weight

Parameter containing:
tensor([[ 0.0170, -0.0198,  0.0068,  ..., -0.0247, -0.0182, -0.0003],
        [-0.0174, -0.0208, -0.0035,  ..., -0.0166,  0.0107,  0.0020],
        [ 0.0250, -0.0136,  0.0134,  ...,  0.0021, -0.0232,  0.0117],
        ...,
        [ 0.0113, -0.0052, -0.0278,  ..., -0.0105, -0.0439, -0.0301],
        [ 0.0277,  0.0181,  0.0075,  ...,  0.0306, -0.0004,  0.0094],
        [-0.0355,  0.0132,  0.0005,  ...,  0.0179,  0.0043, -0.0103]],
       requires_grad=True)

- Initialize the BART model with cnn_dm pretrained model
- Define a function to copy layers from the pretrained BART model to the BART_Extended model
- Make sure that the architecture is properly created

https://github.com/huggingface/transformers/blob/master/src/transformers/models/bart/modeling_bart.py

In [30]:
for pretrained_bart.model.decoder.layers

BartDecoderLayer(
  (self_attn): BartAttention(
    (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (encoder_attn): BartAttention(
    (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
    (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (encoder_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=1024, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=1024, bias=True)
  (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwis

In [ ]:
class BART_Extended(nn.Module):
    def __init__(self):
        super(BART_Extended, self).__init__()
        self.encoder_source = pretrained_bart.model.encoder.copy()
        self.encoder_knowledge = pretrained_bart.model.encoder.copy()
        self.decoder =
        BART_model.model

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [25]:
pretrained_bart.model.encoder.copy()


BartEncoder(
  (embed_tokens): Embedding(50264, 1024, padding_idx=1)
  (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
  (layers): ModuleList(
    (0): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1024, out_features=4096, bias=True)
      (fc2): Linear(in_features=4096, out_features=1024, bias=True)
      (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
    (1): BartEncoderLayer(
      (self_attn): BartAttention(
        (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
        (v_proj): Linear(in_features=1

In [ ]:
# from transformers import BartForConditionalGeneration, BartTokenizer
#
# model = BartForConditionalGeneration.from_pretrained("facebook/bart-large")#, force_bos_token_to_be_generated=True)
# tok = BartTokenizer.from_pretrained("facebook/bart-large")
# example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
# batch = tok(example_english_phrase, return_tensors='pt')
# generated_ids = model.generate(batch['input_ids'])
#assert tok.batch_decode(generated_ids, skip_special_tokens=True) == ['UN Chief Says There Is No Plan to Stop Chemical Weapons in Syria']

In [ ]:
# tok.batch_decode(generated_ids, skip_special_tokens=True)

